In [1]:
#!pip install -e git+https://github.com/casperbroch/ai-economist@stockmarket#egg=ai-economist

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import time

In [3]:
# Define the configuration of the environment that will be built

env_config_dict = {
    # ===== SCENARIO CLASS =====
    # Which Scenario class to use: the class's name in the Scenario Registry (foundation.scenarios).
    # The environment object will be an instance of the Scenario class.
    'scenario_name': 'stock_market_simulation',
    
    # ===== COMPONENTS =====
    # Which components to use (specified as list of ("component_name", {component_kwargs}) tuples).
    #   "component_name" refers to the Component class's name in the Component Registry (foundation.components)
    #   {component_kwargs} is a dictionary of kwargs passed to the Component class
    # The order in which components reset, step, and generate obs follows their listed order below.
    'components': [
        # (1) Building houses
        ('BuyOrSellStocks', {}),
        # (2) Trading collectible resources
        ('ExecCircuitBreaker', {}),
    ],
    

    # ===== STANDARD ARGUMENTS ======
    # kwargs that are used by every Scenario class (i.e. defined in BaseEnvironment)
    'n_agents': 4,          # Number of non-planner agents (must be > 1)
    'world_size': [1, 1], # [Height, Width] of the env world
    'episode_length': 100, # Number of timesteps per episode
    
    # In multi-action-mode, the policy selects an action for each action subspace (defined in component code).
    # Otherwise, the policy selects only 1 action.
    'multi_action_mode_agents': False,
    'multi_action_mode_planner': False,
    
    # When flattening observations, concatenate scalar & vector observations before output.
    # Otherwise, return observations with minimal processing.
    'flatten_observations': False,
    # When Flattening masks, concatenate each action subspace mask into a single array.
    # Note: flatten_masks = True is required for masking action logits in the code below.
    'flatten_masks': True,
    'dense_log_frequency': 1
}

In [4]:
from rllib.env_wrapper import RLlibEnvWrapper
env_obj = RLlibEnvWrapper({"env_config_dict": env_config_dict}, verbose=True)

Inside covid19_components.py: 0 GPUs are available.
No GPUs found! Running the simulation on a CPU.
Inside covid19_env.py: 0 GPUs are available.
No GPUs found! Running the simulation on a CPU.


C:\Users\caspe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\caspe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\caspe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\caspe\AppData\Roaming\Python\Python37\site-

agent reward:  0.595165045951845
agent reward:  1.0
agent reward:  0.553073619819603
agent reward:  0.9699254984868743
planner reward:  1.0
[EnvWrapper] Spaces
[EnvWrapper] Obs (a)   
BuyOrSellStocks-stock_price: (1,)
action_mask    : (21,)
time           : (1,)
world-Endogenous-AbleToBuy: (1,)
world-Endogenous-AbleToSell: (1,)
world-Endogenous-AvailableFunds: (1,)
world-Endogenous-Demand: (1,)
world-Endogenous-Labor: (1,)
world-Endogenous-NumberOfStocks: (1,)
world-Endogenous-StockPrice: (1,)
world-Endogenous-StockPriceHistory: (100,)
world-Endogenous-StocksLeft: (1,)
world-Endogenous-Supply: (1,)
world-Endogenous-TotalBalance: (1,)
world-Endogenous-Volumes: (100,)


[EnvWrapper] Obs (p)   
action_mask    : (2,)
time           : (1,)
world-Prices_History: (100,)
world-Total_Demand: (1,)
world-Total_Supply: (1,)
world-Volumes  : (100,)


[EnvWrapper] Action (a) Discrete(21)
[EnvWrapper] Action (p) Discrete(2)


In [5]:
import ray
from ray.rllib.agents.ppo import PPOTrainer

In [6]:
policies = {
    "a": (
        None,  # uses default policy
        env_obj.observation_space,
        env_obj.action_space,
        {}  # define a custom agent policy configuration.
    ),
    "p": (
        None,  # uses default policy
        env_obj.observation_space_pl,
        env_obj.action_space_pl,
        {}  # define a custom planner policy configuration.
    )
}

# In foundation, all the agents have integer ids and the social planner has an id of "p"
policy_mapping_fun = lambda i: "a" if str(i).isdigit() else "p"

policies_to_train = ["a", "p"]

In [7]:
trainer_config = {
    "multiagent": {
        "policies": policies,
        "policies_to_train": policies_to_train,
        "policy_mapping_fn": policy_mapping_fun,
    }
}

In [8]:
trainer_config.update(
    {
        "num_workers": 2,
        "num_envs_per_worker": 2,
        # Other training parameters
        "train_batch_size":  4000,
        "sgd_minibatch_size": 4000,
        "num_sgd_iter": 1
    }
)

In [9]:
# We also add the "num_envs_per_worker" parameter for the env. wrapper to index the environments.
env_config = {
    "env_config_dict": env_config_dict,
    "num_envs_per_worker": trainer_config.get('num_envs_per_worker'),   
}

trainer_config.update(
    {
        "env_config": env_config        
    }
)

In [10]:
# Initialize Ray
ray.init(webui_host="127.0.0.1")

2024-05-03 10:20:16,953	INFO resource_spec.py:212 -- Starting Ray with 4.59 GiB memory available for workers and up to 2.31 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2024-05-03 10:20:18,456	INFO services.py:1165 -- View the Ray dashboard at 127.0.0.1:8265


{'node_ip_address': '192.168.1.41',
 'raylet_ip_address': '192.168.1.41',
 'redis_address': '192.168.1.41:6379',
 'object_store_address': 'tcp://127.0.0.1:63514',
 'raylet_socket_name': 'tcp://127.0.0.1:64322',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\caspe\\AppData\\Local\\Temp\\ray\\session_2024-05-03_10-20-16_942609_16728'}

In [11]:
# Create the PPO trainer.
trainer = PPOTrainer(
    env=RLlibEnvWrapper,
    config=trainer_config,
    )

2024-05-03 10:20:23,857	ERROR syncer.py:46 -- Log sync requires rsync to be installed.
2024-05-03 10:20:23,861	INFO trainer.py:585 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2024-05-03 10:20:23,862	INFO trainer.py:612 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


agent reward:  0.9753437118245103
agent reward:  0.8737585221382482
agent reward:  1.0
agent reward:  0.8638233651045629
planner reward:  1.0
agent reward:  0.75034604328897
agent reward:  0.8602335303174682
agent reward:  0.7431828666409536
agent reward:  1.0
planner reward:  1.0
(pid=9944) C:\Users\caspe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=9944)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=9944) C:\Users\caspe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=9944)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=9944) C:\Users\caspe\AppData\Roaming\P

(pid=6908) Inside covid19_components.py: 0 GPUs are available.
(pid=6908) No GPUs found! Running the simulation on a CPU.
(pid=6908) Inside covid19_env.py: 0 GPUs are available.
(pid=6908) No GPUs found! Running the simulation on a CPU.
(pid=9944) Inside covid19_components.py: 0 GPUs are available.
(pid=9944) No GPUs found! Running the simulation on a CPU.
(pid=9944) Inside covid19_env.py: 0 GPUs are available.
(pid=9944) No GPUs found! Running the simulation on a CPU.
(pid=6908) agent reward:  1.0
(pid=6908) agent reward:  0.6443111885204135
(pid=6908) agent reward:  0.925444354714466
(pid=6908) agent reward:  0.6788522463599084
(pid=6908) planner reward:  1.0
(pid=9944) agent reward:  0.9304921207245731
(pid=9944) agent reward:  0.8175078933588881
(pid=9944) agent reward:  0.7237781120478213
(pid=9944) agent reward:  1.0
(pid=9944) planner reward:  1.0


2024-05-03 10:20:36,195	INFO trainable.py:181 -- _setup took 12.336 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [12]:
NUM_ITERS = 10
start_train = time.time()

for iteration in range(NUM_ITERS):
    print(f'********** Iter : {iteration} **********')
    start = time.time()
    result = trainer.train()
    length = time.time() - start
    print(f'''episode_reward_mean: {result.get('episode_reward_mean')}''')
    print(f'''it_time_taken: {length}''')

    
length_train = time.time() - start_train
print("Training took", length_train, " seconds.")

********** Iter : 0 **********
(pid=6908) agent reward:  0.7424093639853993
(pid=6908) agent reward:  0.9127210393312177
(pid=6908) agent reward:  0.8642813947045876
(pid=6908) agent reward:  1.0
(pid=6908) planner reward:  1.0
(pid=9944) agent reward:  0.9439251941555353
(pid=9944) agent reward:  0.7524753150956048
(pid=9944) agent reward:  0.850130056046837
(pid=9944) agent reward:  1.0
(pid=9944) planner reward:  1.0
(pid=9944) agent reward:  0.6971938401995664
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.6851729574236387
(pid=9944) agent reward:  0.8418676770701939
(pid=9944) planner reward:  1.0
(pid=6908) agent reward:  0.8109941040846143
(pid=6908) agent reward:  1.0
(pid=6908) agent reward:  0.5417580087768544
(pid=6908) agent reward:  0.8477512971091811
(pid=6908) planner reward:  1.0
(pid=9944) agent reward:  0.6530456037370213
(pid=9944) agent reward:  0.814167010360692
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.6682103692524122
(pid=9944) planner

(pid=9944) planner reward:  -46.11360514450612
(pid=9944) agent reward:  0.6443177098434547
(pid=9944) agent reward:  0.5653133875202139
(pid=9944) agent reward:  0.6502692179204267
(pid=9944) agent reward:  1.0
(pid=9944) planner reward:  -52.76741605187174
(pid=9944) agent reward:  0.5990225968494741
(pid=9944) agent reward:  0.5697711470575809
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.6298541455693956
(pid=9944) planner reward:  -41.82627541837545
(pid=9944) agent reward:  0.5465551577748097
(pid=9944) agent reward:  0.5765335042721121
(pid=9944) agent reward:  0.6631755044268148
(pid=9944) agent reward:  1.0
(pid=9944) planner reward:  -52.82420512832539
(pid=9944) agent reward:  0.5990225968494741
(pid=9944) agent reward:  0.5697711470575809
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.6298541455693956
(pid=9944) planner reward:  -39.00101833380356
(pid=9944) agent reward:  0.5403735722844885
(pid=9944) agent reward:  0.4468484875257021
(pid=9944) agen

(pid=9944) agent reward:  0.778561769492701
(pid=9944) agent reward:  0.535428275831019
(pid=9944) agent reward:  0.6228002275639726
(pid=9944) agent reward:  1.0
(pid=9944) planner reward:  -59.27819679069657
(pid=9944) agent reward:  0.3552143395605204
(pid=9944) agent reward:  0.4403862228470863
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.5096342392879261
(pid=9944) planner reward:  -37.039208678592885
(pid=9944) agent reward:  0.778561769492701
(pid=9944) agent reward:  0.535428275831019
(pid=9944) agent reward:  0.6228002275639726
(pid=9944) agent reward:  1.0
(pid=9944) planner reward:  -58.447747425514194
(pid=9944) agent reward:  0.3552143395605204
(pid=9944) agent reward:  0.4403862228470863
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.5096342392879261
(pid=9944) planner reward:  -39.28367371546238
(pid=9944) agent reward:  0.778561769492701
(pid=9944) agent reward:  0.535428275831019
(pid=9944) agent reward:  0.6228002275639726
(pid=9944) agent rewa

(pid=9944) agent reward:  0.7594682238077036
(pid=9944) agent reward:  0.8675517685281589
(pid=9944) agent reward:  0.7604002109569802
(pid=9944) agent reward:  1.0
(pid=9944) planner reward:  -65.93211695064007
(pid=9944) agent reward:  0.47079554925575146
(pid=9944) agent reward:  0.3193512998323921
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.5839576707932019
(pid=9944) planner reward:  -35.273459338433206
(pid=9944) agent reward:  0.7594682238077036
(pid=9944) agent reward:  0.8675517685281589
(pid=9944) agent reward:  0.7604002109569802
(pid=9944) agent reward:  1.0
(pid=9944) planner reward:  -65.73178424894807
(pid=9944) agent reward:  0.48390173912564216
(pid=9944) agent reward:  0.32044858932597337
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.5872454607049589
(pid=9944) planner reward:  -35.72469827704188
(pid=9944) agent reward:  0.7503775866782553
(pid=9944) agent reward:  0.8997041160043177
(pid=9944) agent reward:  0.7567674245943945
(pid=9944) ag

(pid=9944) agent reward:  0.8765308254018958
(pid=9944) agent reward:  0.9610330425790434
(pid=9944) agent reward:  0.7482891545522095
(pid=9944) agent reward:  1.0
(pid=9944) planner reward:  -63.120114861228956
(pid=9944) agent reward:  0.48063672758235354
(pid=9944) agent reward:  0.32370861961070446
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.5904772442514148
(pid=9944) planner reward:  -35.65533847482286
(pid=9944) agent reward:  0.8765308254018958
(pid=9944) agent reward:  0.9610330425790434
(pid=9944) agent reward:  0.7482891545522095
(pid=9944) agent reward:  1.0
(pid=9944) planner reward:  -61.50921948364678
(pid=9944) agent reward:  0.48083510847712324
(pid=9944) agent reward:  0.3203921551093576
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.625977717571054
(pid=9944) planner reward:  -35.07466549465805
(pid=9944) agent reward:  0.7314373702490309
(pid=9944) agent reward:  0.9214316766030625
(pid=9944) agent reward:  0.7334386423112166
(pid=9944) age

...

(pid=6908) agent reward:  1.0
(pid=6908) agent reward:  0.34183194304526704
(pid=6908) agent reward:  0.5639009716039595
(pid=6908) agent reward:  0.13346597627107326
(pid=6908) planner reward:  -24.15177632764696
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.634214382478205
(pid=9944) agent reward:  0.793255217941325
(pid=9944) agent reward:  0.23474301714854523
(pid=9944) planner reward:  -59.86288754112926
(pid=6908) agent reward:  1.0
(pid=6908) agent reward:  0.7504700339982174
(pid=6908) agent reward:  0.41527464760671345
(pid=6908) agent reward:  0.6812004152839396
(pid=6908) planner reward:  -35.00689568039804
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.3994345329495098
(pid=9944) agent reward:  0.47920574753175615
(pid=9944) agent reward:  0.6424461892241586
(pid=9944) planner reward:  -59.47901922986554
(pid=6908) agent reward:  1.0
(pid=6908) agent reward:  0.34183194304526704
(pid=6908) agent reward:  0.5639009716039595
(pid=6908) agent reward:  0.

(pid=9944) planner reward:  -58.53817525151816
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.31266857030404677
(pid=9944) agent reward:  0.42216848819107905
(pid=9944) agent reward:  0.6820258709706518
(pid=9944) planner reward:  -78.92046702602048
(pid=6908) agent reward:  1.0
(pid=6908) agent reward:  0.3458296740928983
(pid=6908) agent reward:  0.5400658089103726
(pid=6908) agent reward:  0.15820780555729128
(pid=6908) planner reward:  -23.923285445341136
(pid=6908) agent reward:  1.0
(pid=6908) agent reward:  0.5942475231365708
(pid=6908) agent reward:  0.548641947329191
(pid=6908) agent reward:  0.597798009037573
(pid=6908) planner reward:  -32.37195779830398
(pid=9944) agent reward:  0.9730721119707294
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.9655429923652391
(pid=9944) agent reward:  0.4002043659561981
(pid=9944) planner reward:  -61.60390729387186
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.31266857030404677
(pid=9944) agent reward:  0

(pid=9944) agent reward:  0.9305669737091657
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.7912257476633615
(pid=9944) agent reward:  0.2958134302534713
(pid=9944) planner reward:  -63.22359855199126
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.2183354466950689
(pid=9944) agent reward:  0.31760571222956213
(pid=9944) agent reward:  0.5562301386499042
(pid=9944) planner reward:  -214.23696473773518
(pid=9944) agent reward:  0.9123627214573377
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.7577505909494501
(pid=9944) agent reward:  0.290633423618759
(pid=9944) planner reward:  -62.604936960975856
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.2183354466950689
(pid=9944) agent reward:  0.31760571222956213
(pid=9944) agent reward:  0.5562301386499042
(pid=9944) planner reward:  -250.69720402555308
(pid=9944) agent reward:  0.9123627214573377
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.7577505909494501
(pid=9944) agent reward:  0.

(pid=9944) agent reward:  0.528013772363991
(pid=9944) agent reward:  0.5355540699768541
(pid=9944) agent reward:  0.7667172056867086
(pid=9944) agent reward:  1.0
(pid=9944) planner reward:  1.0
(pid=6908) agent reward:  0.957153807803021
(pid=6908) agent reward:  1.0
(pid=6908) agent reward:  0.7423637098696891
(pid=6908) agent reward:  0.575249421543763
(pid=6908) planner reward:  1.0
(pid=9944) agent reward:  0.7982523744734477
(pid=9944) agent reward:  0.7967762010722239
(pid=9944) agent reward:  1.0
(pid=9944) agent reward:  0.8906723016277691
(pid=9944) planner reward:  1.0
(pid=6908) agent reward:  0.6839963005356181
(pid=6908) agent reward:  0.9811171154982933
(pid=6908) agent reward:  0.8064286988530073
(pid=6908) agent reward:  1.0
(pid=6908) planner reward:  1.0


KeyboardInterrupt: 

In [ ]:
#env_obj.env.get_agent(3).state

In [ ]:
def generate_rollout_from_current_trainer_policy(
    trainer, 
    env_obj,
    num_dense_logs=5
):
    dense_logs = {}
    for idx in range(num_dense_logs):
        # Set initial states
        agent_states = {}
        for agent_idx in range(env_obj.env.n_agents):
            agent_states[str(agent_idx)] = trainer.get_policy("a").get_initial_state()
        planner_states = trainer.get_policy("p").get_initial_state()   

        # Play out the episode
        obs = env_obj.reset(force_dense_logging=True)
        for t in range(env_obj.env.episode_length):
            actions = {}
            for agent_idx in range(env_obj.env.n_agents):
                # Use the trainer object directly to sample actions for each agent
                actions[str(agent_idx)] = trainer.compute_action(
                    obs[str(agent_idx)], 
                    agent_states[str(agent_idx)], 
                    policy_id="a",
                    full_fetch=False
                )

            # Action sampling for the planner
            actions["p"] = trainer.compute_action(
                obs['p'], 
                planner_states, 
                policy_id='p',
                full_fetch=False
            )

            obs, rew, done, info = env_obj.step(actions)        
            if done['__all__']:
                break
        dense_logs[idx] = env_obj.env.dense_log
    return dense_logs

In [ ]:
dense_logs = generate_rollout_from_current_trainer_policy(
    trainer, 
    env_obj,
    num_dense_logs=1
)

In [ ]:
#dense_logs[0]['states'][0]

In [ ]:
#planner_gr_score_importances = [log["p"]["GreenScoreImportance"] for log in dense_logs[0]['states']]

In [ ]:
agent_0_green_scores = [log["0"]["endogenous"]["TotalBalance"] for log in dense_logs[0]['states']]
agent_1_green_scores = [log["1"]["endogenous"]["TotalBalance"] for log in dense_logs[0]['states']]
agent_2_green_scores = [log["2"]["endogenous"]["TotalBalance"] for log in dense_logs[0]['states']]
agent_3_green_scores = [log["3"]["endogenous"]["TotalBalance"] for log in dense_logs[0]['states']]

stock_prices = [log["0"]["endogenous"]["StockPrice"] for log in dense_logs[0]['states']]

In [ ]:
plt.plot(np.arange(0,101),agent_0_green_scores, label="Agent 0")
plt.plot(np.arange(0,101),agent_1_green_scores, label="Agent 1")
plt.plot(np.arange(0,101),agent_2_green_scores, label="Agent 2")
plt.plot(np.arange(0,101),agent_3_green_scores, label="Agent 3")
plt.title('Stock Brocker Total Balance')
plt.legend()
plt.xlabel('Days')
plt.ylabel('Total Balance')
#plt.savefig("miners_green_scores.png")
plt.show()

In [ ]:
plt.plot(np.arange(0,101),stock_prices, label="Agent 0")
plt.title('Stock Price Over Time')
plt.legend()
plt.xlabel('Days')
plt.ylabel('Stock Price')
#plt.savefig("miners_green_scores.png")
plt.show()

In [ ]:
stocks_left = [log["0"]["endogenous"]["StocksLeft"] for log in dense_logs[0]['states']]

plt.plot(np.arange(0,101),stocks_left, label="Stocks Quantity Left")
plt.title('Stock Price Over Time')
plt.legend()
plt.xlabel('Days')
plt.ylabel('Stock Price')
#plt.savefig("miners_green_scores.png")
plt.show()

In [ ]:
# Shutdown Ray after use
ray.shutdown()